In [1]:
import openai

In [2]:
client = openai.Client()

In [3]:
assistant = client.beta.assistants.create(
    name="Tutor de tecnologia",
    instructions="Voce é um tutor de tecnologia.",
    tools=[{"type":"code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

In [5]:
pergunta = """
    Em um sistema de cache com mapeamento direto, dado uma memoria principal de uma cache de 256 KB e blocos de 64 bytes:
    - Quantas linhas existem na cache?
    - Onde o bloco de endereço de Ox1A23C será mapeado na cache?
"""

In [ ]:
 # criacao da thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [7]:
# executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [9]:
run.status

'queued'

In [ ]:
# aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [13]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_WsUOGW1uWRe480WfAhKdgaPa', assistant_id='asst_dozU1TWdLvwpU9CmYcSUSZBD', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.\n\nSe precisar de mais alguma coisa, por favor me avise.'), type='text')], created_at=1743640250, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Su4XgX8ciJEW2x0LqnA7SofC', status=None, thread_id='thread_XWxPiz28mXMLJa9qRFlAJbRa'), Message(id='msg_pn79rng6F3UhekjtlZprkEJF', assistant_id='asst_dozU1TWdLvwpU9CmYcSUSZBD', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Os cálculos corrigidos indicam que o bloco de endereço 0x1A23C será dividido da seguinte forma:\n- 6 bits para o offset\n- 12 bits para a linha\n- 14 bits para a tag\n\nAgora, vamos determinar em qual linha da cache o bloco de endereço 0x1A23

In [14]:
print(mensagens.data[0].content[0].text.value)

O bloco de endereço 0x1A23C será mapeado na linha 1672 da cache.

Se precisar de mais alguma coisa, por favor me avise.


In [15]:
# analizando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [17]:
for step in run_steps.data[::-1]:
    print(f"\n=== Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("===========")
            print(tool_call.code_interpreter.input)
            print("===========")
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )

        print(message.content[0].text.value)


=== Step message_creation
Para resolver essa questão, precisamos seguir os seguintes passos:

1. Calcular o número de linhas na cache.
2. Identificar onde o bloco de endereço 0x1A23C será mapeado na cache.

Primeiramente, vamos calcular o número de linhas na cache. Para isso, vamos usar a fórmula:

Número de linhas = Tamanho da Cache / Tamanho do Bloco

Vamos realizar esses cálculos.

=== Step tool_calls
# Tamanho da cache em KB
tamanho_cache_kb = 256

# Tamanho do bloco em bytes
tamanho_bloco_bytes = 64

# Convertendo o tamanho da cache para bytes
tamanho_cache_bytes = tamanho_cache_kb * 1024

# Calculando o número de linhas na cache
numero_linhas_cache = tamanho_cache_bytes / tamanho_bloco_bytes
numero_linhas_cache

=== Step message_creation
A cache terá 4096 linhas.

Agora, vamos identificar onde o bloco de endereço 0x1A23C será mapeado na cache. Para isso, precisamos entender como é feito o mapeamento direto.

No mapeamento direto, o bloco de endereço é dividido em três partes: ta